In [1]:
import os
import numpy as np 

from utils.re_referencing import rref_REST, rref_remove, rref_CSD, rref_average
from utils.data_filtering import preprocess_data

from metrics import analyse_network_by_metric

DEBUG:root:test


In [2]:
# Params load data
data_folder = "../data/"

# Metric
metric = "TE"
targets = None #[16, 17, 18]

# Params Reconstruction
reconst_threshold = None #0.2

# Params Cleaning
clean_threshold = 0.6
window_time_cleaning = 10
rref_fn = rref_REST

# Params AIS
max_lag = 20
min_lag = 1
cmi_estimator = 'JidtGaussianCMI' #'JidtKraskovCMI' 'OpenCLKraskovCMI' 'JidtGaussianCMI'
local_values = False

settings = {
    'cmi_estimator': cmi_estimator, 
    'local_values': local_values, 
    'max_lag_sources': max_lag,
    'min_lag_sources': min_lag,
     #'n_perm_max_stat': 11,
     #'n_perm_min_stat': 11,
     #'alpha_max_stat': 0.1,
     #'n_perm_omnibus': 11,
     #'alpha_omnibus': 0.1,
     #'n_perm_max_seq': 11,
     #'alpha_max_seq': 0.1, 
     #'alpha_min_stat': 0.1,
     'verbose': False
}

save_folder = os.path.join(
    "../results/", 
    f"{metric}_reconstth-{reconst_threshold}_cleanth-{clean_threshold}_windowtimecleaning-{window_time_cleaning}_{rref_fn.__name__}_maxlag-{max_lag}_minlag-{min_lag}_cmiestimator-{cmi_estimator}".replace(".", "o")
)

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [ ]:
hc_filenames = list(filter(lambda x: x.startswith("h"), sorted(os.listdir(data_folder))))
sz_filenames = list(filter(lambda x: x.startswith("s"), sorted(os.listdir(data_folder))))

filenames = sz_filenames
#filenames =  hc_filenames + sz_filenames
filenames

In [ ]:
channels = np.arange(19)

for filename in filenames:
    print("Preprocessing:", filename)
    filepath = os.path.join(data_folder, filename)
    data_windows = preprocess_data(filepath, 
                                   reconst_threshold=reconst_threshold,
                                   clean_threshold=clean_threshold,
                                   window_time=window_time_cleaning,
                                   rref_fn=rref_fn
                                  )
    
    save_subject_folder = filename.split(".edf")[0]
    if not os.path.exists(os.path.join(save_folder, save_subject_folder)):
        os.makedirs(os.path.join(save_folder, save_subject_folder))
    if targets:
        for i_target in targets:
            te_arr = []
            for i_window, data_window in enumerate(data_windows):
                print(f"Working on: {filename} for target: {i_target} with window: {i_window}")
                # Calculating TE
                net_analysis = analyse_network_by_metric(data_window, 
                                                         channels=channels, 
                                                         metric=metric, 
                                                         target=i_target,
                                                         **settings)


                te_arr.append(net_analysis)
                # Store
                te_save = np.array(te_arr)
                save_filename = f"te_itarget-{i_target}"
                save_filepath = os.path.join(save_folder, save_subject_folder, save_filename)
                np.savez(f"{save_filepath}.npz",
                        te=te_save)
    else:
        te_arr = []
        for i_window, data_window in enumerate(data_windows):
            print(f"Working on: {filename} for all data with window: {i_window}")
            # Calculating TE
            net_analysis = analyse_network_by_metric(data_window, 
                                                     channels=channels, 
                                                     metric=metric,
                                                     **settings)


            te_arr.append(net_analysis)
            # Store
            te_save = np.array(te_arr)
            save_filename = "te_all"
            save_filepath = os.path.join(save_folder, save_subject_folder, save_filename)
            np.savez(f"{save_filepath}.npz",
                    te=te_save)

Preprocessing: s01.edf
Extracting EDF parameters from /home/jarajaras/dynamicinfo-eeg/data/s01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 211249  =      0.000 ...   844.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 8251 samples (33.004 sec)



INFO:root:Please use a band in filter_bands=dict_keys(['alpha', 'beta', 'gamma', 'theta'])
INFO:root:window number 33 rejected
INFO:root:window number 38 rejected
INFO:root:window number 41 rejected
INFO:root:window number 46 rejected
INFO:root:window number 49 rejected
INFO:root:window number 50 rejected
INFO:root:window number 60 rejected
INFO:root:window number 79 rejected
INFO:root:window number 80 rejected
INFO:root:window number 83 rejected


Working on: s01.edf for all data with window: 0
Adding data with properties: 19 processes, 2500 samples, 1 replications
overwriting existing data

---------------------------- (1) include target candidates


---------------------------- (2) include source candidates


In [ ]:
#net_analysis.get_adjacency_matrix("max_te_lag", fdr=False).get_edge_list()
#plt.matshow(net_analysis.get_adjacency_matrix("max_p_lag", fdr=False)._weight_matrix)
#net_analysis.get_adjacency_matrix("max_p_lag", fdr=False)._weight_matrix